# Deploy Fine-Tuned Models with Amazon SageMaker AI

# 🚀 Multi-Model Deployment with Amazon SageMaker Inference Components

---

Welcome to this comprehensive guide on deploying multiple fine-tuned models using **Amazon SageMaker Real-Time Inference Endpoint**! 🎯



**🤖 SageMaker Models**: These are machine learning model artifacts that contain the trained model data, inference code, and configuration needed for making predictions. Think of them as packaged ML models ready for deployment.

**🌐 SageMaker Endpoints**: These are fully-managed inference endpoints that host your models and provide real-time prediction capabilities via HTTPS requests. They handle scaling, load balancing, and high availability automatically.

**⚙️ Inference Components**: A powerful feature that allows you to deploy multiple models on a single endpoint, enabling efficient resource utilization and cost optimization. Each component can have its own compute allocation and scaling configuration.

---

In this notebook, we'll demonstrate how to deploy **multiple fine-tuned models** that were trained on Amazon SageMaker AI. We'll show you how to:
- Create and configure multiple SageMaker models
- Set up inference components for efficient multi-model hosting
- Deploy everything to a single, cost-effective endpoint
- Test our deployed models with real inference requests

---


In [ ]:
%pip install -Uq "datasets==4.3.0" \
    "sagemaker==2.253.1"

In [ ]:
import sagemaker
from sagemaker.session import Session
import boto3
import json

In [ ]:
sagemaker_session = Session()
sagemaker_session_bucket = None

if sagemaker_session_bucket is None and sagemaker_session is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sagemaker_session.default_bucket()

try:
    role = sagemaker.session.get_execution_role()
except ValueError:
    iam = boto3.client("iam")
    role = iam.get_role(RoleName="sagemaker_execution_role")["Role"]["Arn"]

sagemaker_session = Session(default_bucket=sagemaker_session_bucket)
bucket_name = sagemaker_session.default_bucket()
default_prefix = sagemaker_session.default_bucket_prefix

sm_client = boto3.client("sagemaker", region_name=sagemaker_session.boto_region_name)
sts = boto3.client("sts", region_name=sagemaker_session.boto_region_name)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sagemaker_session.default_bucket()}")
print(f"sagemaker session region: {sagemaker_session.boto_region_name}")

## Attach to a Completed Training Job

In [ ]:
from sagemaker.estimator import Estimator

pytorch_estimator = Estimator.attach(
    training_job_name="meta-llama--Llama-3-2-3B-Instruct-finetune-20251118054934"
)

s3_model_data_uri = pytorch_estimator.model_data
print(f"Fine-tuned model location: {s3_model_data_uri}")

### Untar the final model weights - `model.tar.gz` and upload the weights to S3

In [ ]:
import os
import boto3
import json
import sagemaker
import tarfile
from getpass import getpass
from datetime import datetime
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.compute_resource_requirements.resource_requirements import ResourceRequirements

In [ ]:
hf_token = getpass()

In [ ]:
local_model_path = "/tmp/tmp_cache_local_model"
os.makedirs(local_model_path, exist_ok=True)

In [ ]:
S3Downloader.download(
    s3_uri=s3_model_data_uri,
    local_path=local_model_path
)
print(f"download model file to {local_model_path}")

In [ ]:
local_gpt_oss_model_path = os.path.join(local_model_path, "gpt-oss-model-finetuned-spectrum")
os.makedirs(local_gpt_oss_model_path, exist_ok=True)


def untar_file(tar_path: str, destination: str) -> None:

    if not os.path.isfile(tar_path):
        raise FileNotFoundError(f"The file '{tar_path}' does not exist.")

    os.makedirs(destination, exist_ok=True)

    with tarfile.open(tar_path, "r:gz") as tar:
        tar.extractall(path=destination)
        print(f"Extracted '{tar_path}' to '{destination}'.")


# untar model file
untar_file(
    tar_path=os.path.join(local_model_path, os.path.basename(s3_model_data_uri)), 
    destination=local_gpt_oss_model_path
)

In [ ]:
model_s3_uri = os.path.join(os.path.dirname(s3_model_data_uri), "full-model-paths")

uploaded_model_s3_uri = S3Uploader.upload(
    local_path=local_gpt_oss_model_path,
    desired_s3_uri=model_s3_uri
)
print(f"Uploaded {local_gpt_oss_model_path} to > {uploaded_model_s3_uri}")

## Deploy as a SageMaker Endpoint

In [ ]:
import time
import sys
from datetime import datetime

In [ ]:
region = sagemaker_session.boto_region_name

inference_image = f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.34.0-lmi16.0.0-cu128"

print(f"inference image: {inference_image}")

In [ ]:
model_name = sagemaker.utils.name_from_base(f"tuned-model")
inference_component_name = f"ic-{model_name}"
endpoint_config_name = f"epc-{model_name}"
endpoint_name = f"ep-{model_name}"

In [ ]:
instance_type = "ml.g6e.2xlarge"
num_gpu = 1
variant_name = "AllTraffic"

In [ ]:
print(f">> Model name: {model_name}")
print(f">> IC name: {inference_component_name}")
print(f">> Endpoint Config name: {endpoint_config_name}")
print(f">> Endpoint name: {endpoint_name}")
print(f">> Instance: {instance_type}")

### Endpoint Configuration

In [ ]:
sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ExecutionRoleArn=role,
    ProductionVariants=[
        {
            "VariantName": variant_name,
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": 3600,
            "ContainerStartupHealthCheckTimeoutInSeconds": 3600,
            "ManagedInstanceScaling": {
                "Status": "ENABLED",
                "MinInstanceCount": 1,
                "MaxInstanceCount": 1,
            },
            "RoutingConfig": {"RoutingStrategy": "LEAST_OUTSTANDING_REQUESTS"},
        }
    ],
)

### SageMaker Endpoint

In [ ]:
sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

In [ ]:
sagemaker_session.wait_for_endpoint(endpoint_name)

In [ ]:
uploaded_model_s3_uri

### Model

In [ ]:
model_configuration = {
    "Image": inference_image,
    'ModelDataSource': {
                'S3DataSource': {
                    'S3Uri': f"{uploaded_model_s3_uri}/",
                    'S3DataType': 'S3Prefix',
                    'CompressionType': 'None',
                }
            },
    "Environment": {
        "SAGEMAKER_MODEL_SERVER_WORKERS": "1",
        "MESSAGES_API_ENABLED": "true",
        "OPTION_MAX_ROLLING_BATCH_SIZE": "8",
        "OPTION_MODEL_LOADING_TIMEOUT": "1500",
        "SERVING_FAIL_FAST": "true",
        "OPTION_ROLLING_BATCH": "disable",
        "OPTION_ASYNC_MODE": "true",
        "OPTION_ENTRYPOINT": "djl_python.lmi_vllm.vllm_async_service",
        "OPTION_ENABLE_STREAMING": "true",
        "HF_TOKEN": hf_token,
        "MAX_TOTAL_TOKENS": json.dumps(4096)
    },
}

In [ ]:
sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    Containers=[model_configuration],
)

### Inference Component

In [ ]:
sm_client.create_inference_component(
    InferenceComponentName=inference_component_name,
    EndpointName=endpoint_name,
    VariantName=variant_name,
    Specification={
        "ModelName": model_name,
        "ComputeResourceRequirements": {
            "NumberOfAcceleratorDevicesRequired": num_gpu,
            "NumberOfCpuCoresRequired": 1,
            "MinMemoryRequiredInMb": 1024,
        },
    },
    RuntimeConfig={"CopyCount": 1},
)

In [ ]:
start_time = time.time()
while True:
    desc = sm_client.describe_inference_component(
        InferenceComponentName=inference_component_name
    )
    status = desc["InferenceComponentStatus"]
    print(status)
    sys.stdout.flush()
    if status in ["InService", "Failed"]:
        break
    time.sleep(30)
total_time = time.time() - start_time
print(f"\nTotal time taken: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

## Run Inference

Invoke your running endpoint with boto3 invoke_endpoint or invoke_endpoint_with_response_stream runtime api calls. If you have an existing endpoint, you don't need to recreate the predictor and can follow below example to invoke the endpoint with an endpoint name.

In [ ]:
import boto3
import json

In [ ]:
sagemaker_runtime = boto3.client('sagemaker-runtime')

In [ ]:
prompt = {
    'messages':[
    {"role": "user", "content": "How many R are in STRAWBERRY? Keep your answer and explanation short!"}
],
    'temperature':0.7,
    'top_p':0.8,
    'top_k':20,
    'max_tokens':512,
}
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    InferenceComponentName=inference_component_name,
    ContentType="application/json",
    Body=json.dumps(prompt)
)

In [ ]:
response_dict = json.loads(response['Body'].read().decode("utf-8"))
response_content = response_dict['choices'][0]['message']['content']
print(response_content)